In [19]:
import importlib

import plotly.graph_objects as go


In [20]:
# Manually reloading python module such that
# jupyter reflects changes without kernel restart

import apollo.api.yahoo_api_connector as yac
import apollo.calculations.models.arima_regression as arc
from apollo.utils.common import to_default_date_string

importlib.reload(yac)
importlib.reload(arc);

In [21]:
from apollo.settings import END_DATE, START_DATE, TICKER


In [22]:
api_connector = yac.YahooApiConnector(
    ticker=str(TICKER),
    start_date=str(START_DATE),
    end_date=str(END_DATE),
)

dataframe = api_connector.request_or_read_prices()

dataframe

,ticker,open,high,low,close,adj close,volume
date,,,,,,,
2007-01-03,SPY,142.250000,142.860001,140.570007,141.369995,101.536606,94807600
2007-01-04,SPY,141.229996,142.050003,140.610001,141.669998,101.752022,69620600
2007-01-05,SPY,141.330002,141.399994,140.380005,140.539993,100.940468,76645300
2007-01-08,SPY,140.820007,141.410004,140.250000,141.190002,101.407318,71655000
2007-01-09,SPY,141.309998,141.600006,140.399994,141.070007,101.321114,75680100
...,...,...,...,...,...,...,...
2012-12-24,SPY,142.479996,142.559998,142.190002,142.350006,116.096169,53874600
2012-12-26,SPY,142.639999,142.710007,141.350006,141.750000,115.606819,106947700
2012-12-27,SPY,141.789993,142.080002,139.919998,141.559998,115.451851,167920600


In [23]:
arc_calculator = arc.ARIMARegressionModelCalculator(
    dataframe=dataframe,
    window_size=5,
)

arc_calculator.forecast_periods()

dataframe

date
2007-01-03           NaN
2007-01-04           NaN
2007-01-05    101.391505
2007-01-08    101.415926
2007-01-09    101.486322
                 ...    
2012-12-24    116.226303
2012-12-26    115.562764
2012-12-27    115.500786
2012-12-28           NaN
2012-12-31           NaN
Name: trend, Length: 1510, dtype: float64


/Users/stankonnov/dev/personal/apollo/venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/Users/stankonnov/dev/personal/apollo/venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/Users/stankonnov/dev/personal/apollo/venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/Users/stankonnov/dev/personal/apollo/venv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



,ticker,open,high,low,close,adj close,volume,arf
date,,,,,,,,
2007-01-03,SPY,142.250000,142.860001,140.570007,141.369995,101.536606,94807600,94.620519
2007-01-04,SPY,141.229996,142.050003,140.610001,141.669998,101.752022,69620600,94.620519
2007-01-05,SPY,141.330002,141.399994,140.380005,140.539993,100.940468,76645300,94.620519
2007-01-08,SPY,140.820007,141.410004,140.250000,141.190002,101.407318,71655000,101.384995
2007-01-09,SPY,141.309998,141.600006,140.399994,141.070007,101.321114,75680100,101.422836
...,...,...,...,...,...,...,...,...
2012-12-24,SPY,142.479996,142.559998,142.190002,142.350006,116.096169,53874600,116.247634
2012-12-26,SPY,142.639999,142.710007,141.350006,141.750000,115.606819,106947700,115.808523
2012-12-27,SPY,141.789993,142.080002,139.919998,141.559998,115.451851,167920600,115.106777


In [24]:
prime_value = "adj close"
trace_value = "arf"

x = dataframe.index.to_numpy()

y1 = dataframe[prime_value].to_numpy()
y2 = dataframe[trace_value].to_numpy()

# Create the first trace with the primary y-axis
trace1 = go.Scatter(x=x, y=y1, name=prime_value)

# Create the second trace with the secondary y-axis
trace2 = go.Scatter(x=x, y=y2, name=f"{trace_value}", yaxis="y2")

# Plot title
title = (
    f"{TICKER}:"
    f" {to_default_date_string(dataframe.index.to_numpy()[0])}"
    f" - {to_default_date_string(dataframe.index.to_numpy()[-1])}"
)

# Create the layout with two y-axes
layout = go.Layout(
    title=title,
    yaxis={},
    yaxis2={"overlaying": "y", "side": "right"},
    height=650,
)

# Create the figure and add traces to it
fig = go.Figure(data=[trace1, trace2], layout=layout)

fig.update_xaxes(
    showspikes=True,
    spikemode="across",
    spikecolor="black",
    spikethickness=0.5,
)